# Note
If you need to run the code there are two options:
- Download all the libraries required in your local machine before running this file.

- Or you can access the google colab link: https://colab.research.google.com/drive/1cl12u6kjZhHFbKGdzjsg5aa2jgQeSbjH?usp=sharing
    In the google colab, go to the folder Icon in the left side, and upload the excel dataset before running the code.


In [ ]:
#Import tensorflow and pandas
import pandas as pd
import tensorflow as tf
#Spell check
!pip install pyspellchecker
from spellchecker import SpellChecker
#nltk & wordnet
!pip install nltk
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet
import re
#Grammer tool
!pip install language_tool_python
import language_tool_python
#SKlearn
from sklearn.model_selection import train_test_split
#Import BERT model and tokenizer
from transformers import TFBertModel, BertTokenizer
# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
# Load the pre-trained BERT model
bert_model = TFBertModel.from_pretrained('bert-base-cased')
#Numpy
import numpy as np

!pip install tensorflow==2.8.0
!pip install transformers==4.18.0


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/502M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
!pip install tensorflow==2.8.0
!pip install transformers==4.18.0

#Data processing

In [ ]:
#Read the excel file
file_path = '/content/ielts_writing_dataset.csv'
data_csv = pd.read_csv(file_path)
data_csv.head() #Check its content


,Task_Type,Question,Essay,Examiner_Commen,Task_Response,Coherence_Cohesion,Lexical_Resource,Range_Accuracy,Overall
0,1,The bar chart below describes some changes abo...,"Between 1995 and 2010, a study was conducted r...",NaN,NaN,NaN,NaN,NaN,5.5
1,2,Rich countries often give money to poorer coun...,Poverty represents a worldwide crisis. It is t...,NaN,NaN,NaN,NaN,NaN,6.5
2,1,The bar chart below describes some changes abo...,The left chart shows the population change hap...,NaN,NaN,NaN,NaN,NaN,5.0
3,2,Rich countries often give money to poorer coun...,Human beings are facing many challenges nowada...,NaN,NaN,NaN,NaN,NaN,5.5
4,1,The graph below shows the number of overseas v...,Information about the thousands of visits from...,NaN,NaN,NaN,NaN,NaN,7.0


In [ ]:
#data_csv['Task_Type'] This represent all the rows in the colm Task_Type, so we want to focus on task 2 only thats why we will filter them this way:

data_task_2 = data_csv[data_csv['Task_Type'] == 2]
#to make sure we did it right lets print data_task_2:
data_task_2.head()

,Task_Type,Question,Essay,Examiner_Commen,Task_Response,Coherence_Cohesion,Lexical_Resource,Range_Accuracy,Overall
1,2,Rich countries often give money to poorer coun...,Poverty represents a worldwide crisis. It is t...,NaN,NaN,NaN,NaN,NaN,6.5
3,2,Rich countries often give money to poorer coun...,Human beings are facing many challenges nowada...,NaN,NaN,NaN,NaN,NaN,5.5
5,2,Some countries achieve international sports by...,Whether countries should only invest facilitie...,NaN,NaN,NaN,NaN,NaN,6.5
7,2,Some countries achieve international sports by...,"Sports is an essential part to most of us , so...",NaN,NaN,NaN,NaN,NaN,5.5
9,2,Some countries achieve international sports by...,International sports events require the most w...,NaN,NaN,NaN,NaN,NaN,9.0


In [ ]:
#------------
#The columns that we need are question, essay and the overall score, we will treat the question and essay columns as features, and the Overall as a label (output).
labels = data_task_2['Overall'].values

features = data_task_2[['Question', 'Essay']].values #adding .values will convert to numpy, features[0] will give the first row for both question and essay columns

# Feature 1- Length

In [ ]:
# Calculate the length of each essay and add it as a new column
data_task_2['Essay_Length'] = data_task_2['Essay'].apply(lambda x: len(x.split()))

# Preview
data_task_2.head()


<ipython-input-6-6c1e82bda7c7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_task_2['Essay_Length'] = data_task_2['Essay'].apply(lambda x: len(x.split()))


,Task_Type,Question,Essay,Examiner_Commen,Task_Response,Coherence_Cohesion,Lexical_Resource,Range_Accuracy,Overall,Essay_Length
1,2,Rich countries often give money to poorer coun...,Poverty represents a worldwide crisis. It is t...,NaN,NaN,NaN,NaN,NaN,6.5,258
3,2,Rich countries often give money to poorer coun...,Human beings are facing many challenges nowada...,NaN,NaN,NaN,NaN,NaN,5.5,298
5,2,Some countries achieve international sports by...,Whether countries should only invest facilitie...,NaN,NaN,NaN,NaN,NaN,6.5,288
7,2,Some countries achieve international sports by...,"Sports is an essential part to most of us , so...",NaN,NaN,NaN,NaN,NaN,5.5,254
9,2,Some countries achieve international sports by...,International sports events require the most w...,NaN,NaN,NaN,NaN,NaN,9.0,269


# Feature 2- Linking words

In [ ]:
# List of most common linking words from https://www.assignmenthelppro.com/blog/linking-words/
linking_words_list = ['as an example', 'results from', 'is the result of', 'third', 'leads to', 'as well', 'to show that', 'concurrently', 'firstly', 'therefore', 'in-line manner', 'while', 'thus', 'in the end',
                      'comes from', 'because', 'earlier', 'in the beginning', 'likewise', 'as a result of', 'moreover', 'furthermore', 'except for', 'once upon a time', 'apart from', 'in the first place', 'yet', 'again', 'by the end', 'despite', 'uniquely', 'finally',
                      'such as', 'an instance', 'even so', 'too', 'as a consequence of', 'at this point', 'secondary', 'briefly', 'in summary', 'as revealed by', 'all in all', 'in short', 'what’s more', 'a similar', 'by and large', 'to conclude', 'although', 'to sum up',
                      'in brief', 'for the most part', 'owing to', 'since', 'then', 'by the large', 'along with', 'or', 'in other words', 'secondly', 'specifically', 'as a result', 'and all', 'still', 'in essence', 'similarly', 'for this reason', 'to begin with', 'afterwords',
                      'in the long run', 'also', 'same as', 'just as', 'last', 'primary', 'previously', 'it all started when', 'along', 'second', 'consequently', 'meanwhile', 'on balance', 'for', 'after all', 'like', 'just as … so too', 'by the same token', 'first', 'in the case of',
                      'to illustrate', 'by the same', 'alternatively', 'hence', 'just like', 'the next step', 'for now', 'in the same way', 'is due to', 'for example', 'in sum', 'stems from', 'in outline', 'overall', 'as with', 'comparable', 'despite this', 'however', 'after', 'nor', 'not only',
                      'but also', 'whereas', 'is caused by', 'and', 'first of all', 'to demonstrate', 'in conclusion', 'following this', 'next', 'in closing', 'for one thing', 'at the same time', 'all things considered', 'in addition', 'thirdly', 'equally', 'causes', 'contributes to', 'evidently',
                      'for instance', 'on the whole', 'to start with', 'so', 'further', 'lastly', 'summarising', 'is the consequence of', 'on the contrary', 'nevertheless', 'besides']


def count_linking_words_percentage(essay, total_words_count, linking_words):
    # Normalize the essay to lowercase to ensure matching works correctly
    essay_lower = essay.lower()

    # Count occurrences of each linking word
    linking_words_count = sum(essay_lower.count(f" {word} ") for word in linking_words)  # Space padding

    # Calculate the percentage of linking words in relation to the total word count
    percentage = (linking_words_count / total_words_count) * 100 if total_words_count > 0 else 0
    return percentage


# .apply() is used with a lambda function to pass both the essay and its length
data_task_2['Linking_Words_Percentage'] = data_task_2.apply(lambda row: count_linking_words_percentage(row['Essay'], row['Essay_Length'], linking_words_list), axis=1)

# Preview
data_task_2.head(100)



<ipython-input-7-ba69ec4e22f8>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_task_2['Linking_Words_Percentage'] = data_task_2.apply(lambda row: count_linking_words_percentage(row['Essay'], row['Essay_Length'], linking_words_list), axis=1)


,Task_Type,Question,Essay,Examiner_Commen,Task_Response,Coherence_Cohesion,Lexical_Resource,Range_Accuracy,Overall,Essay_Length,Linking_Words_Percentage
1,2,Rich countries often give money to poorer coun...,Poverty represents a worldwide crisis. It is t...,NaN,NaN,NaN,NaN,NaN,6.5,258,5.038760
3,2,Rich countries often give money to poorer coun...,Human beings are facing many challenges nowada...,NaN,NaN,NaN,NaN,NaN,5.5,298,4.697987
5,2,Some countries achieve international sports by...,Whether countries should only invest facilitie...,NaN,NaN,NaN,NaN,NaN,6.5,288,7.638889
7,2,Some countries achieve international sports by...,"Sports is an essential part to most of us , so...",NaN,NaN,NaN,NaN,NaN,5.5,254,7.086614
9,2,Some countries achieve international sports by...,International sports events require the most w...,NaN,NaN,NaN,NaN,NaN,9.0,269,7.063197
...,...,...,...,...,...,...,...,...,...,...,...
191,2,Write about the following topic.Some people th...,I believe students should be divided according...,NaN,NaN,NaN,NaN,NaN,7.5,340,7.058824
193,2,Write about the following topic.Some people th...,"Nowadays, people hold different ideas about ho...",NaN,NaN,NaN,NaN,NaN,8.0,421,7.838480
195,2,Write about the following topic.Some people be...,There are a lot of different opinions regardin...,NaN,NaN,NaN,NaN,NaN,7.0,346,6.069364
197,2,Write about the following topic.Some people be...,There is no agreement between supporters of th...,NaN,NaN,NaN,NaN,NaN,7.0,262,6.488550


Feature 3- Unique words counter

In [ ]:
def unique_words_percentage(essay, total_words_count):
    words = essay.split()  # Split the essay into words
    unique_words = set(words)  # Use a set to find unique words
    unique_words_count = len(unique_words)  # Count of unique words
    percentage = (unique_words_count / total_words_count) * 100 if total_words_count > 0 else 0
    return percentage

# .apply() is used with a lambda function to pass both the essay text and its pre-calculated length
data_task_2['Unique_Words_Percentage'] = data_task_2.apply(lambda row: unique_words_percentage(row['Essay'], row['Essay_Length']), axis=1)

# Preview
data_task_2.head()



<ipython-input-8-081621680dfe>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_task_2['Unique_Words_Percentage'] = data_task_2.apply(lambda row: unique_words_percentage(row['Essay'], row['Essay_Length']), axis=1)


,Task_Type,Question,Essay,Examiner_Commen,Task_Response,Coherence_Cohesion,Lexical_Resource,Range_Accuracy,Overall,Essay_Length,Linking_Words_Percentage,Unique_Words_Percentage
1,2,Rich countries often give money to poorer coun...,Poverty represents a worldwide crisis. It is t...,NaN,NaN,NaN,NaN,NaN,6.5,258,5.038760,62.015504
3,2,Rich countries often give money to poorer coun...,Human beings are facing many challenges nowada...,NaN,NaN,NaN,NaN,NaN,5.5,298,4.697987,60.402685
5,2,Some countries achieve international sports by...,Whether countries should only invest facilitie...,NaN,NaN,NaN,NaN,NaN,6.5,288,7.638889,60.763889
7,2,Some countries achieve international sports by...,"Sports is an essential part to most of us , so...",NaN,NaN,NaN,NaN,NaN,5.5,254,7.086614,54.330709
9,2,Some countries achieve international sports by...,International sports events require the most w...,NaN,NaN,NaN,NaN,NaN,9.0,269,7.063197,53.159851


Feature 4- Spelling errors

In [ ]:

def spelling_error_percentage(essay, total_words_count):
    spell = SpellChecker()

    # Tokenize the essay into words
    words = essay.split()

    # Find those words that misspelled
    misspelled = spell.unknown(words)

    # Calculate the percentage of misspelled words
    percentage = (len(misspelled) / total_words_count) * 100 if total_words_count > 0 else 0
    return percentage
# Apply the function to each essay in the DataFrame
data_task_2['Spelling_Error_Percentage'] = data_task_2.apply(lambda row: spelling_error_percentage(row['Essay'], row['Essay_Length']), axis=1)

# Preview
print(data_task_2[['Essay', 'Spelling_Error_Percentage']].head())


                                               Essay  \
1  Poverty represents a worldwide crisis. It is t...   
3  Human beings are facing many challenges nowada...   
5  Whether countries should only invest facilitie...   
7  Sports is an essential part to most of us , so...   
9  International sports events require the most w...   

   Spelling_Error_Percentage  
1                  13.178295  
3                  13.087248  
5                  12.847222  
7                   6.692913  
9                   7.806691  


<ipython-input-9-decaec526f43>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_task_2['Spelling_Error_Percentage'] = data_task_2.apply(lambda row: spelling_error_percentage(row['Essay'], row['Essay_Length']), axis=1)


Feature 5- Synonyms used

In [ ]:
def get_synonyms(word):  # Get the synonyms of a word
    synonyms = set()
    for syn in wordnet.synsets(word):  # For each set of synonyms (synset) that the word belongs to
        for lemma in syn.lemmas():  # for each synonym
            synonyms.add(lemma.name().replace('_', ' ')) # add that synonym (lemma)
    return synonyms

def percentage_used_synonyms(essay, total_words_count):
    words = re.findall(r'\w+', essay.lower())  # Tokenize the essay into words, lowercased
    unique_words = set(words)  # Find unique words in the essay
    used_synonyms_count = 0
    for word in unique_words:
        synonyms = get_synonyms(word)  # Get synonyms for each unique word
        for synonym in synonyms:
            if synonym in unique_words and synonym != word:  # Check if the synonym is used in the essay
                used_synonyms_count += 1
                break  # Once a match is found, move to the next word to avoid double counting

    # Calculate the percentage based on the total words count
    percentage = (used_synonyms_count / total_words_count) * 100 if total_words_count > 0 else 0
    return percentage



data_task_2['Used_Synonyms_Percentage'] = data_task_2.apply(lambda row: percentage_used_synonyms(row['Essay'], row['Essay_Length']), axis=1)

# Preview
data_task_2.head()


<ipython-input-10-3fd4f8952189>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_task_2['Used_Synonyms_Percentage'] = data_task_2.apply(lambda row: percentage_used_synonyms(row['Essay'], row['Essay_Length']), axis=1)


,Task_Type,Question,Essay,Examiner_Commen,Task_Response,Coherence_Cohesion,Lexical_Resource,Range_Accuracy,Overall,Essay_Length,Linking_Words_Percentage,Unique_Words_Percentage,Spelling_Error_Percentage,Used_Synonyms_Percentage
1,2,Rich countries often give money to poorer coun...,Poverty represents a worldwide crisis. It is t...,NaN,NaN,NaN,NaN,NaN,6.5,258,5.038760,62.015504,13.178295,12.403101
3,2,Rich countries often give money to poorer coun...,Human beings are facing many challenges nowada...,NaN,NaN,NaN,NaN,NaN,5.5,298,4.697987,60.402685,13.087248,16.442953
5,2,Some countries achieve international sports by...,Whether countries should only invest facilitie...,NaN,NaN,NaN,NaN,NaN,6.5,288,7.638889,60.763889,12.847222,15.972222
7,2,Some countries achieve international sports by...,"Sports is an essential part to most of us , so...",NaN,NaN,NaN,NaN,NaN,5.5,254,7.086614,54.330709,6.692913,8.267717
9,2,Some countries achieve international sports by...,International sports events require the most w...,NaN,NaN,NaN,NaN,NaN,9.0,269,7.063197,53.159851,7.806691,15.613383


Feature 6- Grammar mistakes

In [ ]:
# Initialize the LanguageTool object
tool = language_tool_python.LanguageTool('en-US')

def grammar_mistakes_percentage(essay, total_words_count):
    # Check the essay for grammar mistakes
    matches = tool.check(essay)
    number_of_mistakes = len(matches)

    # Calculate the percentage of grammar mistakes
    percentage = (number_of_mistakes / total_words_count) * 100 if total_words_count > 0 else 0
    return percentage

data_task_2['Grammar_Mistakes_Percentage'] = data_task_2.apply(lambda row: grammar_mistakes_percentage(row['Essay'], row['Essay_Length']), axis=1)

# Preview
data_task_2.head()

INFO:language_tool_python.download_lt:Unzipping /tmp/tmpselfvg1s.zip to /root/.cache/language_tool_python.
INFO:language_tool_python.download_lt:Downloaded https://www.languagetool.org/download/LanguageTool-6.4.zip to /root/.cache/language_tool_python.
<ipython-input-11-d9cd8788a218>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_task_2['Grammar_Mistakes_Percentage'] = data_task_2.apply(lambda row: grammar_mistakes_percentage(row['Essay'], row['Essay_Length']), axis=1)


,Task_Type,Question,Essay,Examiner_Commen,Task_Response,Coherence_Cohesion,Lexical_Resource,Range_Accuracy,Overall,Essay_Length,Linking_Words_Percentage,Unique_Words_Percentage,Spelling_Error_Percentage,Used_Synonyms_Percentage,Grammar_Mistakes_Percentage
1,2,Rich countries often give money to poorer coun...,Poverty represents a worldwide crisis. It is t...,NaN,NaN,NaN,NaN,NaN,6.5,258,5.038760,62.015504,13.178295,12.403101,1.162791
3,2,Rich countries often give money to poorer coun...,Human beings are facing many challenges nowada...,NaN,NaN,NaN,NaN,NaN,5.5,298,4.697987,60.402685,13.087248,16.442953,4.362416
5,2,Some countries achieve international sports by...,Whether countries should only invest facilitie...,NaN,NaN,NaN,NaN,NaN,6.5,288,7.638889,60.763889,12.847222,15.972222,2.777778
7,2,Some countries achieve international sports by...,"Sports is an essential part to most of us , so...",NaN,NaN,NaN,NaN,NaN,5.5,254,7.086614,54.330709,6.692913,8.267717,8.267717
9,2,Some countries achieve international sports by...,International sports events require the most w...,NaN,NaN,NaN,NaN,NaN,9.0,269,7.063197,53.159851,7.806691,15.613383,1.115242


#Spliting the dataset to three different sets

60% Training set

20% Cross-validation set

20% Test set

In [ ]:
from sklearn.model_selection import train_test_split


# Combine text features, using SEP to indicate a question-answer relation
data_task_2['combined_text'] = data_task_2['Question'] + " [SEP] " + data_task_2['Essay']

# Define features and labels
features = data_task_2[['combined_text', 'Essay_Length', 'Linking_Words_Percentage', 'Unique_Words_Percentage', 'Spelling_Error_Percentage', 'Used_Synonyms_Percentage', 'Grammar_Mistakes_Percentage']]
labels = data_task_2['Overall']

# First split to separate out the test set
X_train_val, X_test, y_train_val, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Second split to separate out the validation set
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42)  # 0.25 * 0.8 = 0.2


<ipython-input-12-429430456a07>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_task_2['combined_text'] = data_task_2['Question'] + " [SEP] " + data_task_2['Essay']


In [ ]:
# preparing training set features
train_text = X_train['combined_text'].tolist()
train_numerical_features = X_train.drop(columns=['combined_text']).to_numpy() # all other colms except for the combined text

#tokenizeing the training set

train_encodings = tokenizer(train_text,
                            add_special_tokens=True,  # Enable [SEP] that we used in the combined_text
                            truncation=True,  # If we above 512 tokens
                            padding=True,  # Add padding to the sequence
                            return_tensors='tf')  # Return format


# Preparation for Validation Set
val_text = X_val['combined_text'].tolist()
val_numerical_features = X_val.drop(columns=['combined_text']).to_numpy()

# Tokenization for Validation Set
val_encodings = tokenizer(val_text,
                          add_special_tokens=True,
                          truncation=True,
                          padding=True,
                          return_tensors='tf')


# Preparation for Testing Set
test_text = X_test['combined_text'].tolist()
test_numerical_features = X_test.drop(columns=['combined_text']).to_numpy()

# Tokenization for Testing Set
test_encodings = tokenizer(test_text,
                           add_special_tokens=True,
                           truncation=True,
                           padding=True,
                           return_tensors='tf')


#Normalization

The reason we normalize is because the scale of the features we added might not be close. If we have a feature that is scaling between 200-500 it will worth more weight that a feature that scaling between 2-5. So we make sure to avoid that by normalizing all the features to one scale.

In [ ]:

all_numerical_features = np.concatenate([train_numerical_features, val_numerical_features, test_numerical_features], axis=0)

from tensorflow.keras.layers.experimental.preprocessing import Normalization

# Create a Normalization layer
normalizer = Normalization(axis=-1)  # Normalize along the last axis

# Adapt the normalizer to the data
normalizer.adapt(all_numerical_features)

# Apply the normalizer to the numerical feature sets
normalized_train_numerical_features = normalizer(train_numerical_features)
normalized_val_numerical_features = normalizer(val_numerical_features)
normalized_test_numerical_features = normalizer(test_numerical_features)



In [ ]:
# Convert tokenized inputs and labels into TensorFlow Datasets
def create_tf_dataset(encodings, numerical_features, labels):
    dataset = tf.data.Dataset.from_tensor_slices(({
        'input_ids': encodings['input_ids'],
        'attention_mask': encodings['attention_mask'],
        'numerical_features': numerical_features
    }, labels))
    return dataset


train_dataset = create_tf_dataset(train_encodings, normalized_train_numerical_features, y_train)
val_dataset = create_tf_dataset(val_encodings, normalized_val_numerical_features, y_val)
test_dataset = create_tf_dataset(test_encodings, normalized_test_numerical_features, y_test)


batch_size = 8
train_dataset = train_dataset.shuffle(10000).batch(batch_size).prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)


In [ ]:
from transformers import TFBertModel, BertTokenizer
import tensorflow as tf

# Load the pre-trained BERT model
bert_model = TFBertModel.from_pretrained('bert-base-cased')

# Ensure the BERT model's pre-trained layers are not trainable to reuse features without modification, this is important for fine tuning
bert_model.trainable = False

# Define model inputs
input_ids = tf.keras.layers.Input(shape=(None,), dtype=tf.int32, name='input_ids')  # Tokens
attention_mask = tf.keras.layers.Input(shape=(None,), dtype=tf.int32, name='attention_mask')
numerical_features_input = tf.keras.layers.Input(shape=(normalized_train_numerical_features.shape[1],), name='numerical_features')  # features

# Use the BERT model
outputs = bert_model(input_ids, attention_mask=attention_mask)

# Take the [CLS] token output
pooled_output = outputs.pooler_output

# Combine BERT pooled output with numerical features
combined_features = tf.keras.layers.Concatenate()([pooled_output, numerical_features_input])

# Add custom layers on top of the specific task
x = tf.keras.layers.Dense(64, activation='relu')(combined_features)  # Dense layer after the custome features added for fine tunning
x = tf.keras.layers.Dropout(0.1)(x)  # 10% of the input units are randomly set to 0, this can prevent overfitting
score = tf.keras.layers.Dense(1, activation=None)(x)  # for a regression task

# Construct the final model
model = tf.keras.Model(inputs=[input_ids, attention_mask, numerical_features_input], outputs=score)

# Model summary to see all layers
model.summary()



Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, None)]       0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, None)]       0           []                               
                                                                                                  
 tf_bert_model_1 (TFBertModel)  TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, None                                           

In [ ]:
# Compile the model with  Adam optimizer, loss function MSE, and metrics MAE
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='mean_squared_error',  # If we use classification, we change this and the metric
              metrics=['mean_absolute_error'])

# Training the model with epochs and batch size as required
epochs = 3
history = model.fit(train_dataset, validation_data=val_dataset, epochs=epochs)


Epoch 1/3
60/60 [==============================] - 1624s 27s/step - loss: 3.9085 - mean_absolute_error: 1.4413 - val_loss: 1.2325 - val_mean_absolute_error: 0.9087
Epoch 2/3
60/60 [==============================] - 1598s 27s/step - loss: 1.4169 - mean_absolute_error: 0.9583 - val_loss: 0.8328 - val_mean_absolute_error: 0.7427
Epoch 3/3
60/60 [==============================] - 1593s 27s/step - loss: 1.3121 - mean_absolute_error: 0.9073 - val_loss: 1.1384 - val_mean_absolute_error: 0.8576


In [ ]:
# Evaluate the model on the test dataset
test_loss, test_mae = model.evaluate(test_dataset)

print(f"Test Loss: {test_loss:.4f}")
print(f"Test Mean Absolute Error (MAE): {test_mae:.4f}")


20/20 [==============================] - 343s 17s/step - loss: 1.3101 - mean_absolute_error: 0.9165
Test Loss: 1.3101
Test Mean Absolute Error (MAE): 0.9165


In [ ]:
# Extract a batch from the test dataset
for batch in test_dataset.take(1):  # You can adjust this for more batches
    inputs, labels = batch

# Predict scores using the model
predicted_scores = model.predict(inputs)

# Print out a few predictions vs actual scores
for i in range(len(predicted_scores)):
    print(f"Essay {i+1}:")
    print(f"Predicted Score: {predicted_scores[i][0]:.2f}")
    print(f"Actual Score: {labels[i].numpy()}\n")


Essay 1:
Predicted Score: 6.20
Actual Score: 5.0

Essay 2:
Predicted Score: 6.19
Actual Score: 7.5

Essay 3:
Predicted Score: 6.13
Actual Score: 8.0

Essay 4:
Predicted Score: 6.38
Actual Score: 6.5

Essay 5:
Predicted Score: 6.65
Actual Score: 9.0

Essay 6:
Predicted Score: 6.75
Actual Score: 6.5

Essay 7:
Predicted Score: 5.83
Actual Score: 5.0

Essay 8:
Predicted Score: 6.58
Actual Score: 6.0



# References:

- I relied on this page to setup the training API:
https://pypi.org/project/easy-tensorflow/1.1.1/


- I got the linking words from this page:
https://www.assignmenthelppro.com/blog/linking-words/

- Dataset from Keggle:
https://www.kaggle.com/datasets/mazlumi/ielts-writing-scored-essays-dataset/data
